# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [2]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from collections import Counter
import pandas as pd
import numpy as np
import nltk
import string
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\demet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\demet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1 - Fase de Preparação dos Dados
---

Carregando o arquivo original de classes.

In [3]:
classes_original = pd.read_excel("dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa.
Essa classes possuem nível 2 na tabela original.

In [4]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
1,33809814,Temas Exclusivos de Pronunciamentos
2,33809634,Meio Ambiente
3,33809514,"Soberania, Defesa Nacional e Ordem Pública"
4,33808912,Política Social
5,33805362,Jurídico
6,33805317,Honorífico
7,33805137,Infraestrutura
8,33769167,Economia e Desenvolvimento
9,33768972,Organização do Estado
10,33685789,Administração Pública


Carrega o dataset contendo as leis que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [5]:
leis_classificadas_original = pd.read_excel("dados/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.xlsx", "Select mvw_u03_prc_doc_tema")
leis_classificadas_original["DES_CLASSE_RAIZ"] = leis_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
leis_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36155183,386343,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36192020,386579,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
2,3,36155185,387419,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36192056,387832,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
4,5,36155187,388197,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as leis (classificadas e não classificadas) e aplica as transformações iniciais.

In [6]:
leis_original = pd.read_excel("dados/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.xlsx", "Select mvw_s01_documento")
leis_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
leis_original.drop(columns="   ", inplace=True)
leis_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set..."
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...
4,32103727,Lei nº 13.988 de 14/04/2020,LEI-13988-2020-04-14,Dispõe sobre a transação nas hipóteses que esp...


In [7]:
leis_original.shape[0]

26959

Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas leis do dataset de leis classificadas.

In [8]:
leis = leis_original.merge(leis_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO","DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
leis = leis.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
leis.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
leis.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,33260515.0,Orçamento Público
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",33808912.0,Política Social
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,33768972.0,Organização do Estado
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33685789.0,Administração Pública
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33809634.0,Meio Ambiente


In [9]:
leis.shape[0]

27743

Limpa os registros duplicados de leis que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [10]:
leis.drop_duplicates(inplace=True)
temp = leis[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
3,36348502,33685789.0,Administração Pública
4,36348502,33809634.0,Meio Ambiente
5,36348502,33769167.0,Economia e Desenvolvimento
20,36062349,33808912.0,Política Social
21,36062349,33769167.0,Economia e Desenvolvimento
...,...,...,...
27349,36032872,33808912.0,Política Social
27350,35556312,33685789.0,Administração Pública
27351,35556312,33808912.0,Política Social
27359,35396946,33809514.0,"Soberania, Defesa Nacional e Ordem Pública"


Tamanho da base tratada de leis.

In [11]:
leis.shape[0]

27294

Total de classificações feitas em leis.

In [12]:
leis_classificadas = leis.query("not COD_CLASSE.isnull()")
quantidade_classificacoes = leis_classificadas.shape[0]
quantidade_classificacoes

17438

Número distinto de leis classificadas.

In [13]:
len(leis_classificadas["COD_DOCUMENTO"].unique())

17103

Quantidade de leis a classificar.

In [14]:
leis_nao_classificadas = leis.query("COD_CLASSE.isnull()")
leis_nao_classificadas.shape[0]

9856

Verificação da distribuição das classes (em %).

In [15]:
distribuicao_classes = leis_classificadas[["COD_CLASSE", "DES_CLASSE"]].groupby("DES_CLASSE").count() / quantidade_classificacoes * 100
distribuicao_classes.sort_values(by="COD_CLASSE", ascending=False)

,COD_CLASSE
DES_CLASSE,
Orçamento Público,64.181672
Infraestrutura,14.382383
Política Social,5.029246
Administração Pública,4.490194
Honorífico,3.847918
Economia e Desenvolvimento,3.693084
Jurídico,2.184884
"Soberania, Defesa Nacional e Ordem Pública",0.951944
Organização do Estado,0.745498


## Desbalanceamento

É possível verificar que há um desbalanceamento grande entre as classes, o que prejudica o processo de aprendizado. Vamos tentar atacá-lo utilizando a técnica SMOTE:

In [16]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from numpy import where


def printClasses(classes):
	counter = Counter(classes)
	for k,v in counter.items():
		per = v / len(classes) * 100
		print('Classe=%s, n=%d (%.3f%%)' % (k, v, per))

X_smote = leis_classificadas.drop(columns=["COD_CLASSE", "DES_CLASSE"]).copy()
y_smote = leis_classificadas["DES_CLASSE"].copy()
printClasses(y_smote)


Classe=Orçamento Público, n=11192 (64.182%)
Classe=Política Social, n=877 (5.029%)
Classe=Organização do Estado, n=130 (0.745%)
Classe=Administração Pública, n=783 (4.490%)
Classe=Meio Ambiente, n=86 (0.493%)
Classe=Economia e Desenvolvimento, n=644 (3.693%)
Classe=Honorífico, n=671 (3.848%)
Classe=Jurídico, n=381 (2.185%)
Classe=Infraestrutura, n=2508 (14.382%)
Classe=Soberania, Defesa Nacional e Ordem Pública, n=166 (0.952%)


## One Class Classifier

Montaremos um exemplo que possa trabalhar as classes separadamente, utilizando o OneClassSVM para predizer se um texto pertence a uma determinada classe de treinamento. No nosso caso, utilizaremos Administração Pública como o caso positivo, e os demais como negativos. O tutorial utilizado como base está aqui: https://www.kaggle.com/code/saikumar587/bbc-text-oneclass-classificaiton



### Preparação

Como Administração Pública será a nossa classe de treinamento, vamos adicionar, no nosso conjunto de treino, uma coluna para dizer se aquele registro faz parte da nossa classe-alvo. Se fizer, o valor armazenado será +1, caso contrário, -1.

A função abaixo tem esse objetivo.


In [17]:
def addPositiveNegativeMarker(dataFrame, columnName, classeAlvo):
    # resultDataFrame = dataFrame.copy()
    # mask = resultDataFrame['DES_CLASSE'].str.equals(classeAlvo)
    # resultDataFrame.loc[mask, columnName] = 1
    dataFrame[columnName] = np.where(dataFrame['DES_CLASSE'] == classeAlvo, 1, -1)

Vamos modificar agora o dataset de treino para incluir essa coluna:

In [18]:
MARKER_COLUMN_NAME = 'MARKER'
CLASSE_TREINAMENTO = 'Administração Pública'
addPositiveNegativeMarker(leis_classificadas, MARKER_COLUMN_NAME, CLASSE_TREINAMENTO)
leis_classificadas.head()

C:\Users\demet\AppData\Local\Temp\ipykernel_8540\907047299.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[columnName] = np.where(dataFrame['DES_CLASSE'] == classeAlvo, 1, -1)


,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE,MARKER
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,33260515.0,Orçamento Público,-1
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",33808912.0,Política Social,-1
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,33768972.0,Organização do Estado,-1
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33685789.0,Administração Pública,1
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33809634.0,Meio Ambiente,-1


Criaremos agora um dataset que contém apenas os dados de Administração Pública:

In [19]:
administracaoPublica_df = leis_classificadas[leis_classificadas[MARKER_COLUMN_NAME] == 1]
administracaoPublica_df.shape

(783, 7)

# TensorFlow

In [52]:
import tensorflow as tf
from tensorflow import keras


In [1]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

# Vectorization parameters
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Limit on the length of text sequences. Sequences longer than this
# will be truncated.
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):
    """Vectorizes texts as sequence vectors.

    1 text = 1 sequence vector with fixed length.

    # Arguments
        train_texts: list, training text strings.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val, word_index: vectorized training and validation
            texts and word index dictionary.
    """
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index

C:\Users\demet\AppData\Roaming\Python\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\demet\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\demet\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ModuleNotFoundError: No module named 'tensorflow.python.keras.preprocessing'

In [ ]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    # Arguments
        num_classes: int, number of classes.

    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

Definiremos os datasets de treino e testes:

In [50]:
X = leis_classificadas['TXT_EMENTA']
y = leis_classificadas[MARKER_COLUMN_NAME]

train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)




## 2 - Estruturação do texto
***

Definição das funções para a tarefa de criação das matrizes de documentos (texto da ementa)

In [37]:
from keras.preprocessing.text import Tokenizer
def kerasTokenize(data):
    tokenizer = Tokenizer(num_words=50,char_level=False)
    tokenizer.fit_on_texts(data)
    return tokenizer.texts_to_matrix(data,mode='count')
    # print(data_encoded)
    # print(tokenizer.word_index)

In [38]:
# matriz_vetores_treino = matriz_vetores(train_data, vocabulario_skipgram)
# matriz_vetores_teste = matriz_vetores(test_data, vocabulario_skipgram)
matriz_vetores_treino = kerasTokenize(train_data)
matriz_vetores_teste = kerasTokenize(test_data)

In [39]:
train_data_processed = keras.preprocessing.sequence.pad_sequences(matriz_vetores_treino,
                                                        value=0.0,
                                                        padding='post',
                                                        maxlen=256)

test_data_processed = keras.preprocessing.sequence.pad_sequences(matriz_vetores_teste,
                                                       value=0.0,
                                                       padding='post',
                                                       maxlen=256)



In [41]:
len(train_data_processed[0]), len(train_data_processed[1])

(256, 256)

In [42]:
print(train_data_processed[0])

[0 6 1 0 2 2 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [43]:
# O formato de entrada é a contagem vocabulário usados pelas avaliações dos filmes (10000 palavras)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [47]:
x_val = train_data_processed[:3000]
partial_x_train = train_data_processed[3000:]

y_val = train_labels[:3000]
partial_y_train = train_labels[3000:]

In [48]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
18/18 [==============================] - 1s 35ms/step - loss: -276.4585 - accuracy: 0.0000e+00 - val_loss: -288.2856 - val_accuracy: 0.0000e+00
Epoch 2/40
18/18 [==============================] - 1s 31ms/step - loss: -296.6894 - accuracy: 0.0000e+00 - val_loss: -309.0489 - val_accuracy: 0.0000e+00
Epoch 3/40
18/18 [==============================] - 0s 27ms/step - loss: -317.7885 - accuracy: 0.0000e+00 - val_loss: -330.7711 - val_accuracy: 0.0000e+00
Epoch 4/40
18/18 [==============================] - 0s 23ms/step - loss: -339.8242 - accuracy: 0.0000e+00 - val_loss: -353.4278 - val_accuracy: 0.0000e+00
Epoch 5/40
18/18 [==============================] - 0s 25ms/step - loss: -362.8402 - accuracy: 0.0000e+00 - val_loss: -376.9627 - val_accuracy: 0.0000e+00
Epoch 6/40
18/18 [==============================] - 0s 25ms/step - loss: -386.7540 - accuracy: 0.0000e+00 - val_loss: -401.4851 - val_accuracy: 0.0000e+00
Epoch 7/40
18/18 [==============================] - 0s 28ms/step - los

In [49]:
results = model.evaluate(test_data,  test_labels, verbose=2)

print(results)

UnimplementedError: Graph execution error:

Detected at node 'sequential_2/Cast' defined at (most recent call last):
    File "c:\Users\demet\anaconda3\envs\mdenv\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\demet\anaconda3\envs\mdenv\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\demet\anaconda3\envs\mdenv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\demet\anaconda3\envs\mdenv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\demet\anaconda3\envs\mdenv\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\demet\anaconda3\envs\mdenv\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\demet\anaconda3\envs\mdenv\lib\async